### CHATBOT USING RASA

Import rasa_core, rasa_nlu spacy and sys



In [73]:
import rasa_core
import rasa_nlu
import spacy

In [74]:
import sys
python = sys.executable

Declaration of intent and sample words/phrases/sentences which refer to the intention decleared, which will be stored inside nlu.md. In other words, these are sample phrases whcih the chatbot will learn from for each intent.

In [76]:
# Writing various intents with examples to nlu.md file

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:thanks
- thanks
- thank you
- appreciated
- gracias


## intent:directions
- how do i get to
- i need to get to
- is there a bus to
- how i go take reach
- I need directions to
- how do i get a cab
- where can i get a taxi
- where can i get a bus
-i need to get somewhere
-I am going somewhere

## intent:choosing_item
- 1
- 2
- 3
- 4
- 5
- 6
- 7

## intent:accomodation_enquiry
- Where is the nearest hotel ?
- is there a hotel near by ?
- shey hotel they this area ?
- Where i fit crash for here ?
- i want to get to a hotel
- i need to get to a hotel
- where can i spend the night ?

## intent: restaurant_enquiry
- i need to get food
- i am hungry
- where is the restaurant ?
- is there a restaurant ?
- where can i get water ?
- where i fit buy food for here ?
- is there a food court nearby ?
- where them they sell food ?
- I want chop
- where is the nearest shop?
- where can i eat ?
- i need to eat 
- are there eatries around ?
- where can i buy snacks ?
- I want to buy food


## intent:toilet_enquiry
- i neeed to pee
- where is the rest room?
- where is the toilet
- is there a nearby toilet?
- i they find toilet
- where is the bathroom
- i need to use the bathroom
- where can i pee
- i want peace
- i am pressed


 

"""


%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


Setting up our configuration for our pipeline and policies

In [83]:
# Adding the NLU components to the pipeline in config.yml file
config = """
language: "en_core_web_md"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms

policies:
- name: "RulePolicy"
  # Confidence threshold for the `core_fallback_action_name` to apply.
  # The action will apply if no other action was predicted with
  # a confidence >= core_fallback_threshold
  core_fallback_threshold: 0.68
  core_fallback_action_name: "action_default_fallback"
  enable_fallback_prediction: True


""" 

%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


# Training the model
Import the load_data and config modules so as to load the training data file and config file respectiely, which would be fed to the Trainer module to train the model.

In [84]:
# Import modules for training
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

# training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

Testing our model.
The aim is to see how well it can predict intents, this would be then use to determind the appropriate response.

In [81]:
 # Testing the NLU model with an input message

import json
def pprint(o):   
    print(json.dumps(o, indent=2))

pprint(interpreter.parse("x"))

{
  "intent": {
    "name": "choosing_item",
    "confidence": 0.2828135698895929
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "choosing_item",
      "confidence": 0.2828135698895929
    },
    {
      "name": "toilet_enquiry",
      "confidence": 0.22107422237132043
    },
    {
      "name": "greet",
      "confidence": 0.18720818935689895
    },
    {
      "name": "restaurant_enquiry",
      "confidence": 0.11261922993313768
    },
    {
      "name": "accomodation_enquiry",
      "confidence": 0.059228491850562066
    },
    {
      "name": "goodbye",
      "confidence": 0.05440423941512719
    },
    {
      "name": "directions",
      "confidence": 0.041882422727887614
    },
    {
      "name": "thanks",
      "confidence": 0.040769634455473235
    }
  ],
  "text": "x"
}


This is the second part of the project where we would be dealing with stories, actions and templates so our bot can respond to queries appropriately. Stories is a path which countains intents and their respective actions to be taken.
Actions are the the responses we want our bot to give after identifying intent.
Templates are where we store all the actions.
Slots are our bots memories which help it keep track of events.

In [95]:
# Writing stories and saving it in the stories.md file

stories_md = """
##  Greeting path             
* greet              
  - utter_greet

## directions path
* directions
  - utter_menu
* choosing_destination
  - utter_destination_received

## Enquires path
* accomodation_enquiry
  - utter_enquiry

## thanks path
* thanks
  - utter_thanks  

## restaurant path
* restaurant_enquiry
  - utter_restaurant_enquiry


## toilet path
* toilet_enquiry
  - utter_toilet_location


## say goodbye
* goodbye
  - utter_goodbye

## thanks
* thanks
  - utter_thanks

## fallback rule
* bot_fallback
  - action_default_fallback

"""

%store stories_md >stories.md

Writing 'stories_md' (str) to file 'stories.md'.


The domain is where we link together all our files for the chatbot. The uttar_default action is he action to be taken when the confidence value of all possible intentions is below the thereshold value set inn our configuration file under the fallback policy.

In [98]:
# Writing all the intents,slots,entities,actions and templates to domain.yml

domain_yml = """
intents:
- greet
- goodbye
- directions
- choosing_item
- accomodation_enquiry
- restaurant_enquiry
- toilet_enquiry
- thanks



slots:
  group:
    type: text

entities:
- group


actions:
- utter_greet
- utter_menu
- utter_destination_received
- utter_enquiry
- utter_toilet_location
- utter_restaurant_enquiry
- utter_goodbye
- utter_thanks
- utter_default


templates:
  utter_greet:
  - text: "\n\n NRC BOT: \n\n Hey! Welcome to RASA. How may I help you ? \n\n \n\n You:"

  utter_menu:
  - text: "\n\n NRC BOT: \n\n There is a bus terminal outside the station, it's just accross the road.  You can board a bus to the following destinations: \n\n 1. Wuse  2. Airport 3. Nyanya  4. Zuba  5. Asokoro  6. Maitama  7. Area one  8. Central area  8. Secretariat. \n\n Enter  the appropraite number to know when the next bus leaves the terminal. \n\n \n\n  You:"

  utter_destination_received:
  - text: "\n\n NRC BOT: \n\n The next bus leaves in 15 minutes, do have a safe trip. \n\n \n\n You:"

  utter_goodbye:
  - text: "\n\n NRC BOT: \n\n Thank you for choosing to transit with us. Waiting for your next visit. \n\n \n\n You:"

  utter_enquiry:
  - text: "\n\n NRC BOT: \n\n Take a right turn from the exit, there are several hotels at your service. \n\n \n\n You:"

  utter_toilet_location:
  - text: "\n\n NRC BOT: \n\n There is a toilet symbol on the extreme right from where you stand. \n\n \n\n You:"

  utter_restaurant_enquiry:
  - text: "\n\n NRC BOT: \n\n There is a food court filled with a variety of rastaurants at the extreme left after the entrance. \n\n \n\n You: "

  utter_thanks:
  - text: "\n\n NRC BOT: \n\n my pleasure \n\n \n\n You:"

  utter_default:
  - text: "\n\n NRC BOT: \n\n I'm sorry, I didn't quite understand that. Could you rephrase? \n\n You:"

 
 
"""

%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Here we import our policies and the Agent module. The Agent module is used to load the file, and passed to the 'agent' model. The 'agent' model is in turn trained with the data in the stories file.

In [99]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent

# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), ])

# loading our  training dialogues from `stories.md` 
training_data = agent.load_data('stories.md')


# Training the model
agent.train(
    training_data)
    #validation_split=0.0,
    #epochs=200)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 9/9 [00:00<00:00, 127.71it/s, # trackers=20]
Processed actions: 19it [00:00, 622.89it/s, # examples=19]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 27)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7680      
_________________________________________________________________
dense (Dense)                (None, 17)                561       
_________________________________________________________________
activation (Activation)      (None, 17)                0         
Total params: 8,241
Trainable params: 8,241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
413/413 [==============================] - 1s 2ms/sample - loss: 2.7170 - acc: 0.3729
Epoch 2/100
413/413 [==============================] - 0s 214us/sample - loss: 2.5180 - acc: 0.5012
Epoch 3/100
413/413 [==============================] - 0s 254us/sample - l

Load the trained model

In [100]:
#Starting the Bot
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

Write a function to tale inputs for the chatbot and gives out an output while stopping when 'stop' is typed in.

In [104]:
print("Your bot is ready to talk! Type your messages here or send 'stop'" + "\n\n" + "You:")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'

You:
Good morning
NRC BOT:
Hey! Welcome to RASA. How may I help you ?
You:
Bye
NRC BOT:
Thank you for choosing to transit with us. Waiting for your next visit.
You:
stop
